In [20]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import tensorflow as tf

%matplotlib inline

In [3]:
def plot_series(time, series, format="-", start=0, end=None):
    plt.plot(time[start:end], series[start:end], format)
    plt.xlabel("Time")
    plt.ylabel("Value")
    plt.grid(True)

def trend(time, slope=0):
    return slope * time

def seasonal_pattern(season_time):
    """Just an arbitrary pattern, you can change it if you wish"""
    return np.where(season_time < 0.4,
                    np.cos(season_time * 2 * np.pi),
                    1 / np.exp(3 * season_time))

def seasonality(time, period, amplitude=1, phase=0):
    """Repeats the same pattern at each period"""
    season_time = ((time + phase) % period) / period
    return amplitude * seasonal_pattern(season_time)

def noise(time, noise_level=1, seed=None):
    rnd = np.random.RandomState(seed)
    return rnd.randn(len(time)) * noise_level

In [4]:
time = np.arange(4 * 365 + 1, dtype="float32")
baseline = 10
series = trend(time, 0.1)  
baseline = 10
amplitude = 40
slope = 0.05
noise_level = 5

# Create the series
series = (baseline 
          + trend(time, slope) 
          + seasonality(time, period=365, amplitude=amplitude))
# Update with noise
series += noise(time, noise_level, seed=42)

split_time = 1000
time_train = time[:split_time]
x_train = series[:split_time]
time_valid = time[split_time:]
x_valid = series[split_time:]

window_size = 20
batch_size = 32
shuffle_buffer_size = 1000

In [5]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    dataset = tf.data.Dataset.from_tensor_slices(series)
    dataset = dataset.window(
        window_size + 1, shift=1, drop_remainder=True)
    dataset = dataset.flat_map(
        lambda window: window.batch(window_size + 1))
    dataset = dataset.shuffle(shuffle_buffer)\
        .map(lambda window: (window[:-1], window[-1]))
    dataset = dataset.batch(batch_size).prefetch(1)
    return dataset

In [6]:
tf.keras.backend.clear_session()
tf.random.set_seed(88)
np.random.seed(88)

In [7]:
train_set = windowed_dataset(x_train, 
                             window_size, 
                             batch_size=128, 
                             shuffle_buffer=shuffle_buffer_size)

In [8]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1), 
                           input_shape=[None]),
    tf.keras.layers.SimpleRNN(40, return_sequences=True),
    tf.keras.layers.SimpleRNN(40),
    tf.keras.layers.Dense(1),
    tf.keras.layers.Lambda(lambda x: x * 100.)])

In [9]:
lr_schedule = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-8 * 10**(epoch / 20))
optimizer = tf.keras.optimizers.SGD(lr=1e-8, momentum=0.9)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda (Lambda)              (None, None, 1)           0         
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, None, 40)          1680      
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 40)                3240      
_________________________________________________________________
dense (Dense)                (None, 1)                 41        
_________________________________________________________________
lambda_1 (Lambda)            (None, 1)                 0         
Total params: 4,961
Trainable params: 4,961
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.compile(
    loss=tf.keras.losses.Huber(), optimizer=optimizer, metrics=['mae'])

In [11]:
history = model.fit(train_set, epochs=100, callbacks=[lr_schedule])

Epoch 1/100
8/8 [==============================] - 3s 354ms/step - loss: 39.1815 - mae: 39.5929
Epoch 2/100
8/8 [==============================] - 0s 57ms/step - loss: 38.5798 - mae: 39.3059
Epoch 3/100
8/8 [==============================] - 0s 59ms/step - loss: 38.4708 - mae: 38.8515
Epoch 4/100
8/8 [==============================] - 0s 59ms/step - loss: 37.3308 - mae: 38.2806
Epoch 5/100
8/8 [==============================] - 0s 59ms/step - loss: 37.0726 - mae: 37.6172
Epoch 6/100
8/8 [==============================] - 0s 59ms/step - loss: 36.3334 - mae: 36.8692
Epoch 7/100
8/8 [==============================] - 0s 59ms/step - loss: 35.7562 - mae: 36.0309
Epoch 8/100
8/8 [==============================] - 0s 59ms/step - loss: 34.4074 - mae: 35.0912
Epoch 9/100
8/8 [==============================] - 0s 59ms/step - loss: 33.7501 - mae: 34.0541
Epoch 10/100
8/8 [==============================] - 0s 59ms/step - loss: 32.5925 - mae: 32.9042
Epoch 11/100
8/8 [==============================

8/8 [==============================] - 1s 63ms/step - loss: 15.3884 - mae: 15.7998
Epoch 88/100
8/8 [==============================] - 1s 63ms/step - loss: 16.8914 - mae: 17.0048
Epoch 89/100
8/8 [==============================] - 1s 63ms/step - loss: 25.3446 - mae: 25.8468
Epoch 90/100
8/8 [==============================] - 1s 64ms/step - loss: 28.1968 - mae: 27.6580
Epoch 91/100
8/8 [==============================] - 1s 64ms/step - loss: 27.7375 - mae: 29.2328
Epoch 92/100
8/8 [==============================] - 1s 63ms/step - loss: 19.2151 - mae: 19.1388
Epoch 93/100
8/8 [==============================] - 1s 63ms/step - loss: 40.2927 - mae: 42.6187
Epoch 94/100
8/8 [==============================] - 0s 62ms/step - loss: 50.4801 - mae: 51.6261
Epoch 95/100
8/8 [==============================] - 1s 63ms/step - loss: 48.0829 - mae: 52.3193
Epoch 96/100
8/8 [==============================] - 0s 62ms/step - loss: 58.6721 - mae: 59.7044
Epoch 97/100
8/8 [==============================] - 0

In [12]:
history.history

{'loss': [39.093955931371575,
  38.80701192739058,
  38.35240243405712,
  37.78138446418607,
  37.11822400774275,
  36.369711755246534,
  35.53221082103496,
  34.59255780589824,
  33.5558072459941,
  32.4056605436364,
  31.136658446642816,
  29.737357509379486,
  28.242810089734135,
  26.64766792764469,
  24.965840997501296,
  23.22436219818738,
  21.543489191483477,
  19.912518512959384,
  18.46868022996552,
  17.246847830013355,
  16.28092003647162,
  15.511071504865374,
  15.012368965148926,
  14.638704416703204,
  14.417562656013333,
  14.199685995919364,
  13.96412256591174,
  13.665293783071089,
  13.291894562390386,
  12.822461914529606,
  12.241939494074607,
  11.59354525196309,
  10.9309488958242,
  10.271549536257375,
  9.698396336302466,
  9.184261711276307,
  8.69007007443175,
  8.313980614409155,
  8.035238016868124,
  7.913739315344363,
  7.763114454308335,
  7.695022863271285,
  7.663397384176449,
  7.617055309062102,
  7.571928164423729,
  7.404122290319326,
  7.3616609

In [13]:
#plt.semilogx(history.history['lr'], history.history['loss'])
#plt.axis([1e-8, 1e-4, 0, 30])

In [14]:
BEST_LEARNING_RATE = 5e-5

In [16]:
optimizer = tf.keras.optimizers.SGD(lr=BEST_LEARNING_RATE, momentum=0.9)
model.compile(loss=tf.keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae"])
history = model.fit(train_set, epochs=400)

Epoch 1/400
8/8 [==============================] - 3s 338ms/step - loss: 125.5237 - mae: 129.0785
Epoch 2/400
8/8 [==============================] - 1s 63ms/step - loss: 64.6519 - mae: 63.8688
Epoch 3/400
8/8 [==============================] - 0s 59ms/step - loss: 40.0987 - mae: 40.2518
Epoch 4/400
8/8 [==============================] - 0s 59ms/step - loss: 22.6741 - mae: 22.8216
Epoch 5/400
8/8 [==============================] - 0s 59ms/step - loss: 18.4574 - mae: 18.6035
Epoch 6/400
8/8 [==============================] - 0s 59ms/step - loss: 12.5227 - mae: 12.7508
Epoch 7/400
8/8 [==============================] - 0s 59ms/step - loss: 8.6880 - mae: 9.2209
Epoch 8/400
8/8 [==============================] - 0s 59ms/step - loss: 7.4520 - mae: 7.8586
Epoch 9/400
8/8 [==============================] - 0s 59ms/step - loss: 7.1876 - mae: 7.7116
Epoch 10/400
8/8 [==============================] - 0s 59ms/step - loss: 6.7177 - mae: 7.1298
Epoch 11/400
8/8 [==============================] - 0s

8/8 [==============================] - 1s 65ms/step - loss: 4.7695 - mae: 5.1850
Epoch 89/400
8/8 [==============================] - 1s 65ms/step - loss: 4.6537 - mae: 5.1634
Epoch 90/400
8/8 [==============================] - 1s 65ms/step - loss: 5.1049 - mae: 5.5411
Epoch 91/400
8/8 [==============================] - 1s 65ms/step - loss: 5.4041 - mae: 5.9537
Epoch 92/400
8/8 [==============================] - 1s 63ms/step - loss: 5.1796 - mae: 5.6074
Epoch 93/400
8/8 [==============================] - 1s 64ms/step - loss: 5.9745 - mae: 6.4117
Epoch 94/400
8/8 [==============================] - 1s 64ms/step - loss: 5.8030 - mae: 6.2227
Epoch 95/400
8/8 [==============================] - 1s 64ms/step - loss: 5.2754 - mae: 5.7530
Epoch 96/400
8/8 [==============================] - 1s 64ms/step - loss: 4.6455 - mae: 5.2003
Epoch 97/400
8/8 [==============================] - 1s 63ms/step - loss: 5.6492 - mae: 6.0765
Epoch 98/400
8/8 [==============================] - 1s 63ms/step - loss: 

8/8 [==============================] - 1s 63ms/step - loss: 4.8613 - mae: 5.3895
Epoch 175/400
8/8 [==============================] - 1s 63ms/step - loss: 5.3381 - mae: 5.7566
Epoch 176/400
8/8 [==============================] - 1s 64ms/step - loss: 5.4318 - mae: 5.8679
Epoch 177/400
8/8 [==============================] - 1s 65ms/step - loss: 5.1559 - mae: 5.6062
Epoch 178/400
8/8 [==============================] - 1s 65ms/step - loss: 6.9504 - mae: 7.5336
Epoch 179/400
8/8 [==============================] - 1s 65ms/step - loss: 6.3440 - mae: 6.9694
Epoch 180/400
8/8 [==============================] - 1s 65ms/step - loss: 5.9073 - mae: 6.4149
Epoch 181/400
8/8 [==============================] - 1s 65ms/step - loss: 5.1474 - mae: 5.5940
Epoch 182/400
8/8 [==============================] - 1s 65ms/step - loss: 5.0071 - mae: 5.5408
Epoch 183/400
8/8 [==============================] - 1s 65ms/step - loss: 4.6727 - mae: 5.1440
Epoch 184/400
8/8 [==============================] - 1s 65ms/ste

8/8 [==============================] - 1s 65ms/step - loss: 6.3093 - mae: 6.6942
Epoch 261/400
8/8 [==============================] - 1s 65ms/step - loss: 5.0121 - mae: 5.4367
Epoch 262/400
8/8 [==============================] - 1s 65ms/step - loss: 4.5430 - mae: 5.1293
Epoch 263/400
8/8 [==============================] - 1s 65ms/step - loss: 4.6461 - mae: 5.1339
Epoch 264/400
8/8 [==============================] - 1s 65ms/step - loss: 4.6380 - mae: 5.1891
Epoch 265/400
8/8 [==============================] - 1s 65ms/step - loss: 5.1462 - mae: 5.6034
Epoch 266/400
8/8 [==============================] - 1s 65ms/step - loss: 5.6990 - mae: 6.1041
Epoch 267/400
8/8 [==============================] - 1s 65ms/step - loss: 4.5758 - mae: 4.9757
Epoch 268/400
8/8 [==============================] - 1s 65ms/step - loss: 4.2913 - mae: 4.7874
Epoch 269/400
8/8 [==============================] - 1s 65ms/step - loss: 4.7444 - mae: 5.1479
Epoch 270/400
8/8 [==============================] - 1s 65ms/ste

8/8 [==============================] - 1s 65ms/step - loss: 6.5686 - mae: 7.0171
Epoch 347/400
8/8 [==============================] - 1s 65ms/step - loss: 4.8853 - mae: 5.2799
Epoch 348/400
8/8 [==============================] - 1s 65ms/step - loss: 4.7791 - mae: 5.3317
Epoch 349/400
8/8 [==============================] - 1s 65ms/step - loss: 5.9871 - mae: 6.3944
Epoch 350/400
8/8 [==============================] - 1s 64ms/step - loss: 5.3746 - mae: 5.8312
Epoch 351/400
8/8 [==============================] - 1s 65ms/step - loss: 4.3936 - mae: 4.8300
Epoch 352/400
8/8 [==============================] - 1s 65ms/step - loss: 4.6519 - mae: 5.1236
Epoch 353/400
8/8 [==============================] - 1s 65ms/step - loss: 4.2964 - mae: 4.7802
Epoch 354/400
8/8 [==============================] - 1s 65ms/step - loss: 4.0955 - mae: 4.6356
Epoch 355/400
8/8 [==============================] - 1s 65ms/step - loss: 4.4174 - mae: 4.9270
Epoch 356/400
8/8 [==============================] - 1s 65ms/ste

In [17]:
forecast = []
for time in range(len(series) - window_size):
    forecast.append(
        model.predict(series[time:time + window_size][np.newaxis]))

forecast = forecast[split_time - window_size:]
results = np.array(forecast)[:, 0, 0]

In [18]:
#plt.figure(figsize=(10, 6))
#plot_series(time_valid, x_valid)
#plot_series(time_valid, results)

In [19]:
tf.keras.metrics.mean_absolute_error(x_valid, results).numpy()

7.1082096

In [21]:
# Retrieve a list of list results on training and test data sets for each
# training epoch
mae = history.history['mae']
loss = history.history['loss']
epochs = range(len(loss))

In [ ]:
plt.plot(epochs, mae, 'r')
plt.plot(epochs, loss, 'b')
plt.title('MAE and Loss')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend(["MAE", "Loss"])

In [22]:
epochs_zoom = epochs[200:]
mae_zoom = mae[200:]
loss_zoom = loss[200:]

In [ ]:
plt.plot(epochs_zoom, mae_zoom, 'r')
plt.plot(epochs_zoom, loss_zoom, 'b')
plt.title('MAE and Loss')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend(["MAE", "Loss"])